In [113]:
%matplotlib inline
import torch

Autograd: Tính đạo hàm tự động
===================================

Về mặt toán học thì mạng neural có thể được xem như một hàm toán học phức tạp, và việc tính đạo hàm của nó là rất quan trọng khi huấn luyện mạng neural.

Vì vậy ``autograd`` hay chức năng tính đạo hàm tự động là một chức năng vô cùng quan trọng của Pytorch. Nói ngắn gọn, thì ``autograd`` giúp ta tính được giá trị đạo hàm tại các điểm cụ thể với hàm số được tạo ra từ mọi phép toán trên Tensor mà không cần phải biết công thức chính xác.

Nhắc lại về đạo hàm
--------

Đạo hàm của một hàm số mô tả sự biến thiên của một hàm số tại một điểm, ví dụ 
cho hàm số $y =3x^2$ thì đạo hàm $y'=6x$.
Tại điểm cụ thể $x = 5$, ta có $y = 75$ và $y' = 30$ (giải thích nôm na là, khi x thay đổi 1 đơn vị, thì y thay đổi 30 đơn vị, trong giới hạn nhỏ quanh điểm $x=5$). Ví dụ tính toán đơn giản này có thể được mô tả trong pytorch như sau:



In [114]:
# Khởi tạo điểm x = 5
x = torch.tensor([5.0], requires_grad=True)
y = 3 * x * x
# Tính y tại điểm x = 5
print(y)

tensor([75.], grad_fn=<ThMulBackward>)


In [115]:
# Tính đạo hàm
y.backward()
# In ra giá trị đạo hàm y' = dy/dx tại điểm x = 5
print(x.grad)

tensor([30.])


Như vậy là pytorch giúp ta tính được giá trị của đạo hàm $y'$ tại một điểm x mà không cần dùng công thức đạo hàm $x^n = nx^{n-1}$

Ta có thể mở rộng ra việc tính đạo hàm với hàm nhiều biến với ví dụ sau, cho hàm số $z = 5x^3 + 2y^2$, ta có đạo hàm riêng $\frac{dz}{dx}=15x^2$, và $\frac{dz}{dy}=4y$. Tại điểm $(x = 7, y = 8)$ ta có $z = 1843$ và $\frac{dz}{dx}=735$, $\frac{dz}{dy}=32$

Ví dụ này được mô tả trong pytorch như sau:

In [116]:
# Khởi tạo điểm x = 5
x = torch.tensor([7.0], requires_grad=True)
y = torch.tensor([8.0], requires_grad=True)
z = 5 * x * x * x + 2 * y * y
# Tính z tại điểm (x = 7, y = 8)
print(z)

tensor([1843.], grad_fn=<ThAddBackward>)


In [117]:
# Tính đạo hàm
z.backward()
# In ra giá trị đạo hàm dz/dx
print(x.grad)

tensor([735.])


In [118]:
# In ra giá trị đạo hàm dz/dy
print(y.grad)

tensor([32.])


Lưu ý trong các ví dụ trên, khi khai báo các tensor như x, y và sau này cần tính đạo hàm, ta cần thêm thuộc tính `requires_grad=True` để pytorch ghi nhớ các phép toán trên các tensor này.

Đạo hàm của Tensor
--------
Ở các ví dụ trên đây, ta đã khai báo x, y như các đại lượng vô hướng (Tensor với kích thước 1), vậy trong trường hợp tổng quát thì sao? Ta hãy quay lại ví dụ đầu tiên nhưng với x là 1 ma trận 2x2:


In [119]:
x = torch.tensor([[3.0, 4.0], [5.0, 2.0]], requires_grad=True)
y = 3 * x * x
print(y)
y = y.sum()
y.backward()
print(x.grad)

tensor([[27., 48.],
        [75., 12.]], grad_fn=<ThMulBackward>)
tensor([[18., 24.],
        [30., 12.]])


Lưu ý, trong ví dụ trên đây, $x*x$ chỉ phép lấy tích hai ma trận theo từng phân tử, chứ không phải là phép nhân ma trận. Ta thấy trong trường hợp này, kết quả đạo hàm $y'$ chỉ là mở rộng của trường hợp đại lượng vô hướng; mỗi phần tử của $y'$ là giá trị đạo hàm tại điểm tương ứng trên ma trận $x$ ban đầu. Ta hãy thử thay $x*x$ bằng phép nhân ma trận (thể hiện qua hàm `torch.mm`) và xem giá trị đạo hàm thay đổi như thế nào. 

In [120]:
x = torch.tensor([[3.0, 4.0], [5.0, 2.0]], requires_grad=True)
y = 3 * torch.mm(x, x)
print(y)
y = y.sum()
y.backward()
print(x.grad)

tensor([[87., 60.],
        [75., 72.]], grad_fn=<MulBackward>)
tensor([[45., 45.],
        [39., 39.]])


Trong ví dụ trên, giả sử
$
x = \left(\begin{array} 
\\a & b\\
c & d
\end{array}\right)
$, thì ta sẽ có $y = 3 * (a^2 + d^2 + ab + ac + bc + bd + cd)$, và đạo hàm riêng $\frac{dy}{da}=6a + 3b + 3c$, với $
x = \left(\begin{array} 
\\3 & 4\\
5 & 2
\end{array}\right)
$ thì  $\frac{dy}{da}=45$

Ghi chú: ``autograd`` cũng hoạt động được khi y không phải là đại lượng vô hướng; tuy nhiên điều này nằm ngoài phạm vi của bài này.

Autograd với hàm hợp
--------
Giá trị của autograd thể hiện rõ thêm với một hàm số hợp được định nghĩa qua nhiều bước (đặc biệt hay áp dụng trong mạng Neural khi kết quả là sự kếp hợp của nhiều lớp Neural). Ví dụ $y=2x^3$ và $z=5y^2$, nếu tính đạo hàm thủ công ta có thể dùng công thức $\frac{dz}{dx}=\frac{dz}{dy}\frac{dy}{dx}=10y6x^2=120x^5$. Tại $x=2$ thì $\frac{dz}{dx}=3840$. Ví dụ này được mô tả qua pytorch như sau:

In [121]:
# Khởi tạo điểm x = 2
x = torch.tensor([2.0], requires_grad=True)
y = 2 * x * x * x
z = 5 * y * y
print(y)
print(z)

tensor([16.], grad_fn=<ThMulBackward>)
tensor([1280.], grad_fn=<ThMulBackward>)


In [122]:
z.backward()
print(x.grad)

tensor([3840.])


Autograd hoạt động như thế nào
--------
Ta có thể giải thích cơ chế hoạt động của autograd từ ví dụ đơn giản trên đây. Autograd sẽ xây dựng hàm số đích dưới dạng một đồ thị tính toán (computation graph) qua từng bước, và cũng áp dụng công thức $\frac{dz}{dx}=\frac{dz}{dy}\frac{dy}{dx}$ để tính ngược đạo hàm. Lưu ý là autograd giúp ta tính giá trị cụ thể của đạo hàm tại một điểm xác định nên công thức này sẽ áp dụng trên giái trị cụ thể, chứ không phải trên công thức tổng quát (symbolic differentiation). Trong ví dụ trên, đầu tiên autograd sẽ tính $\frac{dz}{dy} = 160$, sau đó tính được $\frac{dy}{dx} = 24$, và cuối cùng $\frac{dz}{dx}=3840$